In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns

In [ ]:
# Load and preprocess the data
def load_data(manhole, tipo):
    label = []
    arr = []
    strr = "/content/drive/MyDrive/Manhole detection/Man_hole_DB/" + manhole + "/" + tipo + "/*"
    for file_ in glob.glob(strr):
        img = cv2.imread(file_)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        arr.append(img)
        label.append(manhole)
    return arr, label

In [ ]:
def whole_train_data(tipo):
    closed_data, closed_label = load_data('Closed', tipo)
    open_data, open_label = load_data('Open', tipo)
    road_data, road_label = load_data('Road', tipo)
    data = np.concatenate((closed_data, open_data, road_data))
    labels = np.concatenate((closed_label, open_label, road_label))
    return data, labels

In [ ]:
data_train, labels_train = whole_train_data('Train')
data_test, labels_test = whole_train_data('Test')
data_train.shape, labels_train.shape

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


((3715,), (3715,))

In [ ]:
# Function to preprocess the images for HOG feature extraction
def preprocessing_part_two(arr):
    arr_feature = []
    for i in range(np.shape(arr)[0]):
        img = arr[i]
        # Resize the image to (72, 72)
        img_resized = resize(img, (72, 72), anti_aliasing=True)
        arr_feature.append(img_resized)
    return arr_feature

In [ ]:
data_train_p = np.array(preprocessing_part_two(data_train))
data_test_p = np.array(preprocessing_part_two(data_test))

In [ ]:
def ExtractHOG(img):
    if img.ndim == 3:  # If the image is color (3D), convert it to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ftr, _ = hog(img, orientations=8, pixels_per_cell=(16, 16),
                 cells_per_block=(1, 1), visualize=True, multichannel=False)
    return ftr


In [ ]:
data_train_ftr = [ExtractHOG(img) for img in data_train_p]
data_test_ftr = [ExtractHOG(img) for img in data_test_p]


error: ignored

In [ ]:
# Train the SVM classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(data_train_ftr, labels_train)

In [ ]:
# Make predictions and calculate accuracy and F1 score
y_pred = svm_clf.predict(data_test_ftr)
accuracy = accuracy_score(labels_test, y_pred)
f1 = f1_score(labels_test, y_pred, average='weighted')
print('Test Accuracy:', accuracy * 100, "%")
print('F1 Score:', f1)

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix(labels_test, y_pred)

In [ ]:
# Define the class names for the labels
class_names = np.unique(labels_train)

In [ ]:
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Display random test image and its predicted label
def showImg(img, name):
    plt.axis("off")
    plt.title(name)
    plt.imshow(img)
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
# Load and preprocess the data
def load_data(manhole, tipo):
    label = []
    arr = []
    strr = "/content/drive/MyDrive/Manhole detection/Man_hole_DB/" + manhole + "/" + tipo + "/*"
    for file_ in glob.glob(strr):
        img = cv2.imread(file_)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        arr.append(img)
        label.append(manhole)
    return arr, label
def whole_train_data(tipo):
    closed_data, closed_label = load_data('Closed', tipo)
    open_data, open_label = load_data('Open', tipo)
    road_data, road_label = load_data('Road', tipo)
    data = np.concatenate((closed_data, open_data, road_data))
    labels = np.concatenate((closed_label, open_label, road_label))
    return data, labels
data_train, labels_train = whole_train_data('Train')
data_test, labels_test = whole_train_data('Test')
data_train.shape, labels_train.shape
# Function to preprocess the images for HOG feature extraction
def preprocessing_part_two(arr):
    arr_feature = []
    for i in range(np.shape(arr)[0]):
        img = arr[i]
        # Resize the image to (72, 72)
        img_resized = resize(img, (72, 72), anti_aliasing=True)
        arr_feature.append(img_resized)
    return arr_feature
data_train_p = np.array(preprocessing_part_two(data_train))
data_test_p = np.array(preprocessing_part_two(data_test))
def ExtractHOG(img):
    if img.ndim == 3:  # If the image is color (3D), convert it to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ftr, _ = hog(img, orientations=8, pixels_per_cell=(16, 16),
                 cells_per_block=(1, 1), visualize=True, multichannel=False)
    return ftr
data_train_ftr = [ExtractHOG(img) for img in data_train_p]
data_test_ftr = [ExtractHOG(img) for img in data_test_p]
# Train the SVM classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(data_train_ftr, labels_train)
# Make predictions and calculate accuracy and F1 score
y_pred = svm_clf.predict(data_test_ftr)
accuracy = accuracy_score(labels_test, y_pred)
f1 = f1_score(labels_test, y_pred, average='weighted')
print('Test Accuracy:', accuracy * 100, "%")
print('F1 Score:', f1)
# Generate the confusion matrix
cm = confusion_matrix(labels_test, y_pred)
# Define the class names for the labels
class_names = np.unique(labels_train)
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
# Display random test image and its predicted label
def showImg(img, name):
    plt.axis("off")
    plt.title(name)
    plt.imshow(img)
    plt.show()
from random import seed
from random import randint
x_ = randint(0, data_test_p.shape[0] - 1)
predicted_label = svm_clf.predict([data_test_ftr[x_]])
predicted_manhole = predicted_label[0]
showImg(data_test[x_], predicted_manhole)

Mounted at /content/drive


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


error: ignored